# ## Install Dependencies
# Install the required Python libraries for this project.


In [ ]:
# Verify installations
import numpy as np
import sklearn
import hazm
import json



# Persian Text Retrieval System
# This notebook implements a simple Persian text retrieval system using the `IR_data_news_12k.json` dataset. The system uses TF-IDF for ranking documents and evaluates performance using precision and recall.

# ## 1. Load the Dataset
# Load the `IR_data_news_12k.json` dataset.


In [4]:
def read_json(path):
  file = open(path)
  data = json.load(file)
  return data

In [5]:
input_data = read_json('./data/IR_data_news_12k.json')

In [6]:
print(list(input_data.values())[0])

{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا', 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n', 'tags': ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند'], 'date': '3/15/2022 5:59:27 PM', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا', 'category': 'sports'}


In [9]:
print(list(input_data.values())[0].keys())
contents = [input_data[i]['content'] for i in input_data]
print(len(contents))
print(contents[0])

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])
12202

به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/





# ## 2. Preprocess the Data
# Use the `Hazm` library to normalize, tokenize, and remove stop words from the text.


In [17]:
from parsivar import Normalizer, Tokenizer, FindStems
from hazm import stopwords_list

In [18]:
normalizer = Normalizer()
tokenizer = Tokenizer()
stemmer = FindStems()

In [19]:
stopwords = {stopwords_list()[i] for i in range(0, len(stopwords_list()) - 1)}
extra_stopwords = ['،', '.', ')', '(', '}', '{', '«', '»', '؛', ':',  '؟','>','<','|','+','-','*',"^",'%','#','=','_','/','«','»','$','[',']','&',"❊",'«','»']
stopwords.update(extra_stopwords)

In [20]:
def preprocess(contents, rm_sw=True, stemming=True):
  preprocessed_docs = []
  for content in contents:
    
    # normalizing
    normalized_content = normalizer.normalize(content)
    content_tokens = tokenizer.tokenize_words(normalized_content)
    tokens = []
    for token in content_tokens:
      # stemming
      if stemming:
        token = stemmer.convert_to_stem(token)
      # remove stopwords
      if rm_sw:
          if token in stopwords:
                continue
      tokens.append(token)
    preprocessed_docs.append(tokens)
    # tokens of each doc
  return preprocessed_docs

In [21]:
preprocessed_docs = preprocess(contents)

In [22]:
print(len(preprocessed_docs))

12202


# ## 2. Positional Indexing